##### The cell below is for you to keep track of the libraries used and install those libraries quickly
##### Ensure that the proper library names are used and the syntax of `%pip install PACKAGE_NAME` is followed

In [1]:
%pip install pandas numpy scikit-learn==1.2.2
# add commented pip installation lines for packages used as shown above for ease of testing
# the line should be of the format %pip install PACKAGE_NAME

Note: you may need to restart the kernel to use updated packages.


## **DO NOT CHANGE** the filepath variable
##### Instead, create a folder named 'data' in your current working directory and
##### have the .parquet file inside that. A relative path *must* be used when loading data into pandas

In [53]:
# Can have as many cells as you want for code
import pandas as pd
import warnings
pd.options.mode.chained_assignment = None  # default='warn'
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pickle
from datetime import datetime

filepath = "./data/catB_train.parquet"
# the initialised filepath MUST be a relative path to a folder named data that contains the parquet file

### **ALL** Code for machine learning and dataset analysis should be entered below.
##### Ensure that your code is clear and readable.
##### Comments and Markdown notes are advised to direct attention to pieces of code you deem useful.

In [3]:
# read in data
df = pd.read_parquet(filepath)

In [47]:
def feature_engineering(df):
    # impute empty values for race and country
    df[['race_desc', 'ctrycode_desc']] = df[['race_desc', 'ctrycode_desc']].fillna(value='Others')

    # convert dates to number of years
    rownum = len(df.index)
    today_date = [datetime.today()] * rownum
    df['today'] = today_date
    df['min_occ_date'].replace('None', '1900-01-01', inplace=True) # set to a very high number
    df['cltdob_fix'].replace('None', '1900-01-01', inplace=True)
    df['min_occ_date']= pd.to_datetime(df['min_occ_date'])
    df['cltdob_fix']= pd.to_datetime(df['cltdob_fix'])
    col_index_1 = df.columns.get_loc('min_occ_date')
    col_index_2 = df.columns.get_loc('cltdob_fix')
    df.insert(loc=col_index_1, column='years_since_last_bought', value=(df['min_occ_date'] - df['today']) / np.timedelta64(365, '1D'))
    df.insert(loc=col_index_2, column='customer_age', value=(df['cltdob_fix'] - df['today']) / np.timedelta64(365, '1D'))
    df['years_since_last_bought'] = df['years_since_last_bought'].round().abs()
    df['customer_age'] = df['customer_age'].round().abs()

    # drop irrelevant columns
    df = df.drop(['clntnum', 'today', 'min_occ_date', 'cltdob_fix'], axis=1)

    # impute -1, 0, 9999 for NA values in lapse_ape, n_months_since, flg_gi_claim, clmcon, hlthclaim and giclaim columns
    lapse_ape_colnames = list(df.filter(regex="lapse_ape").columns)
    n_months_since_colnames = list(df.filter(regex="n_months_since").columns)
    flg_gi_claim_colnames = list(df.filter(regex="flg_gi_claim").columns)
    clmcon_colnames = list(df.filter(regex="clmcon").columns)
    hlthclaim_colnames = list(df.filter(regex="hlthclaim").columns)
    giclaim_colnames = list(df.filter(regex="giclaim").columns)

    df[lapse_ape_colnames] = df[lapse_ape_colnames].fillna(value=0)
    df[n_months_since_colnames] = df[n_months_since_colnames].fillna(value=9999)
    df[flg_gi_claim_colnames] = df[flg_gi_claim_colnames].fillna(value=0)
    df[clmcon_colnames] = df[clmcon_colnames].fillna(value=-1)
    df[hlthclaim_colnames] = df[hlthclaim_colnames].fillna(value=-1)
    df[giclaim_colnames] = df[giclaim_colnames].fillna(value=-1)

    # impute 0 for these columns
    columns_to_impute = ['tot_cancel_pols', 'recency_lapse', 'recency_cancel', 'flg_affconnect_show_interest_ever', 'flg_affconnect_ready_to_buy_ever', 'affcon_visit_days']
    df[columns_to_impute] = df[columns_to_impute].fillna(value=0)

    # impute -1 for these flg_affconnect_lapse_ever columns
    df['flg_affconnect_lapse_ever'] = df['flg_affconnect_lapse_ever'].fillna(value=-1)

    # drop columns with only one value
    nunique = df.nunique()
    cols_to_drop = nunique[nunique == 1].index
    df = df.drop(cols_to_drop, axis=1)

    # split dataset into 'P' group and 'C' or 'G' group
    df_p = df[df['clttype'] == 'P']
    df_cg = df[(df['clttype'] == 'C') | (df['clttype'] == 'G')]

    # 'P' group specific feature engineering
    # drop specific rows with na values
    df_p = df_p.dropna(subset=['hh_size', 'annual_income_est'])
    n_months_colnames = list(df_p.filter(regex="n_months").columns)
    for col in n_months_colnames:
      df_p[col] = df_p[col].astype(int)
      df_p.drop(df_p[df_p[col] < 0].index, inplace = True)

    # drop hh_20, pop_20, hh_size_est column, clttype and round hh_size column
    df_p = df_p.drop(['hh_20', 'pop_20', 'hh_size_est', 'clttype'], axis=1)

    # round hh_size column
    df_p['hh_size'] = df_p['hh_size'].round()

    # 'C' or 'G' group specific feature engineering
    # drop specific rows with -ve values
    n_months_colnames = list(df_cg.filter(regex="n_months").columns)
    for col in n_months_colnames:
      df_cg[col] = df_cg[col].astype(int)
      df_cg.drop(df_cg[df_cg[col] < 0].index, inplace = True)

    # drop hh_20, pop_20, hh_size_est column, hh_size, annual_income_est, race_desc and round hh_size column
    df_cg = df_cg.drop(['hh_20', 'pop_20', 'hh_size_est', 'hh_size', 'annual_income_est', 'race_desc'], axis=1)

    # fill remaining rows with mode
    for column in df_p.columns:
      df_p[column].fillna(df_p[column].mode()[0], inplace=True)
    df_p.reset_index()

    for column in df_cg.columns:
      df_cg[column].fillna(df_cg[column].mode()[0], inplace=True)
    df_cg.reset_index()

    # one-hot encoding of categorical variables
    df_p_ohe = pd.get_dummies(df_p, columns = ['stat_flag', 'race_desc', 'ctrycode_desc', 'cltsex_fix', 'annual_income_est'])
    df_cg_ohe = pd.get_dummies(df_cg, columns = ['stat_flag', 'clttype', 'ctrycode_desc', 'cltsex_fix'])

    # drop columns from xgboost feature engineering
    df_p_ohe = df_p_ohe[['is_valid_dm', 'is_valid_email', 'is_class_1_2', 'n_months_last_bought_gi', 'f_mindef_mha', 'f_retail']]
    df_cg_ohe = df_cg_ohe[['years_since_last_bought', 'customer_age', 'flg_substandard', 'flg_is_borderline_standard',
                           'flg_is_revised_term', 'flg_is_rental_flat', 'flg_has_health_claim', 'flg_has_life_claim',
                           'flg_gi_claim', 'flg_is_proposal', 'flg_with_preauthorisation', 'flg_is_returned_mail', 'is_consent_to_mail',
                           'is_consent_to_email', 'is_consent_to_call', 'is_consent_to_sms', 'is_valid_dm', 'is_valid_email',
                           'is_housewife_retiree', 'is_sg_pr', 'is_class_1_2', 'n_months_last_bought_products', 'flg_latest_being_lapse',
                           'flg_latest_being_cancel', 'recency_lapse', 'recency_cancel', 'tot_inforce_pols', 'tot_cancel_pols',
                           'ape_grp_6fc3e6', 'ape_grp_de05ae', 'ape_grp_945b5a', 'ape_grp_6a5788', 'ape_ltc_43b9d5', 'ape_grp_9cdedf',
                           'ape_grp_1581d7', 'ape_grp_22decf', 'ape_lh_507c37', 'ape_lh_839f8a', 'ape_inv_e9f316', 'ape_grp_caa6ff',
                           'ape_grp_fd3bfb', 'ape_lh_e22a6a', 'ape_grp_70e1dd', 'ape_grp_e04c3a', 'ape_grp_fe5fb8', 'ape_grp_94baec',
                           'ape_grp_e91421', 'ape_lh_f852af', 'ape_lh_947b15', 'sumins_grp_6fc3e6', 'sumins_grp_de05ae', 'sumins_grp_945b5a',
                           'sumins_grp_6a5788', 'sumins_ltc_43b9d5', 'sumins_grp_9cdedf', 'sumins_grp_1581d7', 'sumins_lh_507c37', 'sumins_inv_e9f316',
                           'sumins_grp_caa6ff', 'sumins_grp_fd3bfb', 'sumins_grp_70e1dd', 'sumins_grp_fe5fb8', 'sumins_grp_e91421', 'sumins_lh_f852af',
                           'sumins_lh_947b15', 'prempaid_grp_6fc3e6', 'prempaid_grp_de05ae', 'prempaid_grp_945b5a', 'prempaid_grp_6a5788',
                           'prempaid_ltc_43b9d5', 'prempaid_grp_9cdedf', 'prempaid_grp_1581d7', 'prempaid_grp_22decf', 'prempaid_lh_507c37',
                           'prempaid_lh_839f8a', 'prempaid_inv_e9f316', 'prempaid_grp_caa6ff', 'prempaid_grp_fd3bfb', 'prempaid_lh_e22a6a',
                           'prempaid_grp_70e1dd', 'prempaid_grp_e04c3a', 'prempaid_grp_fe5fb8', 'prempaid_grp_94baec', 'prempaid_grp_e91421',
                           'prempaid_lh_f852af', 'prempaid_lh_947b15', 'ape_839f8a', 'ape_e22a6a', 'ape_c4bda5', 'ape_ltc', 'ape_507c37',
                           'f_hold_839f8a', 'f_hold_e22a6a', 'f_hold_c4bda5', 'f_hold_ltc', 'f_hold_507c37', 'sumins_839f8a', 'sumins_c4bda5',
                           'sumins_ltc', 'sumins_507c37', 'prempaid_839f8a', 'prempaid_e22a6a', 'prempaid_c4bda5', 'prempaid_ltc', 'prempaid_507c37',
                           'lapse_ape_grp_6fc3e6', 'lapse_ape_grp_de05ae', 'lapse_ape_grp_945b5a', 'lapse_ape_grp_6a5788', 'lapse_ape_ltc_43b9d5',
                           'lapse_ape_grp_9cdedf', 'lapse_ape_grp_1581d7', 'lapse_ape_grp_22decf', 'lapse_ape_lh_507c37', 'lapse_ape_lh_839f8a',
                           'lapse_ape_inv_e9f316', 'lapse_ape_grp_caa6ff', 'lapse_ape_grp_fd3bfb', 'lapse_ape_lh_e22a6a', 'lapse_ape_grp_70e1dd',
                           'lapse_ape_grp_e04c3a', 'lapse_ape_grp_fe5fb8', 'lapse_ape_grp_94baec', 'lapse_ape_grp_e91421', 'lapse_ape_lh_f852af',
                           'lapse_ape_lh_947b15', 'n_months_since_lapse_grp_6fc3e6', 'n_months_since_lapse_grp_de05ae', 'n_months_since_lapse_grp_945b5a',
                           'n_months_since_lapse_grp_6a5788', 'n_months_since_lapse_ltc_43b9d5', 'n_months_since_lapse_grp_9cdedf',
                           'n_months_since_lapse_grp_1581d7', 'n_months_since_lapse_grp_22decf', 'n_months_since_lapse_lh_507c37',
                           'n_months_since_lapse_lh_839f8a', 'n_months_since_lapse_inv_e9f316', 'n_months_since_lapse_grp_caa6ff',
                           'n_months_since_lapse_grp_fd3bfb', 'n_months_since_lapse_lh_e22a6a', 'n_months_since_lapse_grp_70e1dd',
                           'n_months_since_lapse_grp_e04c3a', 'n_months_since_lapse_grp_fe5fb8', 'n_months_since_lapse_grp_94baec',
                           'n_months_since_lapse_grp_e91421', 'n_months_since_lapse_lh_f852af', 'n_months_since_lapse_lh_947b15',
                           'f_ever_bought_839f8a', 'f_ever_bought_e22a6a', 'f_ever_bought_c4bda5', 'f_ever_bought_ltc', 'f_ever_bought_507c37',
                           'f_ever_bought_gi', 'n_months_last_bought_839f8a', 'n_months_last_bought_e22a6a', 'n_months_last_bought_c4bda5',
                           'n_months_last_bought_ltc', 'n_months_last_bought_507c37', 'n_months_last_bought_gi', 'f_ever_bought_grp_6fc3e6',
                           'f_ever_bought_grp_de05ae', 'f_ever_bought_grp_945b5a', 'f_ever_bought_grp_6a5788', 'f_ever_bought_ltc_43b9d5',
                           'f_ever_bought_grp_9cdedf', 'f_ever_bought_grp_1581d7', 'f_ever_bought_grp_22decf', 'f_ever_bought_lh_507c37',
                           'f_ever_bought_lh_839f8a', 'f_ever_bought_inv_e9f316', 'f_ever_bought_grp_caa6ff', 'f_ever_bought_grp_fd3bfb',
                           'f_ever_bought_lh_e22a6a', 'f_ever_bought_grp_70e1dd', 'f_ever_bought_grp_e04c3a', 'f_ever_bought_grp_fe5fb8',
                           'f_ever_bought_grp_94baec', 'f_ever_bought_grp_e91421', 'f_ever_bought_lh_f852af', 'f_ever_bought_lh_947b15',
                           'n_months_last_bought_grp_6fc3e6', 'n_months_last_bought_grp_de05ae', 'n_months_last_bought_grp_945b5a',
                           'n_months_last_bought_grp_6a5788', 'n_months_last_bought_ltc_43b9d5', 'n_months_last_bought_grp_9cdedf',
                           'n_months_last_bought_grp_1581d7', 'n_months_last_bought_grp_22decf', 'n_months_last_bought_lh_507c37',
                           'n_months_last_bought_lh_839f8a', 'n_months_last_bought_inv_e9f316', 'n_months_last_bought_grp_caa6ff',
                           'n_months_last_bought_grp_fd3bfb', 'n_months_last_bought_lh_e22a6a', 'n_months_last_bought_grp_70e1dd',
                           'n_months_last_bought_grp_e04c3a', 'n_months_last_bought_grp_fe5fb8', 'n_months_last_bought_grp_94baec',
                           'n_months_last_bought_grp_e91421', 'n_months_last_bought_lh_f852af', 'n_months_last_bought_lh_947b15',
                           'n_months_last_bought_32c74c', 'f_elx', 'f_mindef_mha', 'f_retail', 'flg_affconnect_show_interest_ever',
                           'flg_affconnect_ready_to_buy_ever', 'flg_affconnect_lapse_ever', 'affcon_visit_days', 'n_months_since_visit_affcon',
                           'clmcon_visit_days', 'recency_clmcon', 'recency_clmcon_regis', 'hlthclaim_amt', 'recency_hlthclaim',
                           'hlthclaim_cnt_success', 'recency_hlthclaim_success', 'hlthclaim_cnt_unsuccess', 'recency_hlthclaim_unsuccess',
                           'flg_hlthclaim_839f8a_ever', 'recency_hlthclaim_839f8a', 'flg_hlthclaim_14cb37_ever', 'recency_hlthclaim_14cb37',
                           'giclaim_amt', 'recency_giclaim', 'stat_flag_ACTIVE', 'stat_flag_LAPSED', 'clttype_C', 'clttype_G',
                           'ctrycode_desc_Others', 'ctrycode_desc_Singapore', 'cltsex_fix_Female', 'cltsex_fix_Male']]

    return df_p_ohe, df_cg_ohe

def load_model():
    filepath_p = "./models/rf_model_p.pkl"
    filepath_cg = "./models/rf_model_cg.pkl"

    # load model from pickle file
    with open(filepath_p, 'rb') as file:
        model_p = pickle.load(file)

    with open(filepath_cg, 'rb') as file:
        model_cg = pickle.load(file)
    
    return model_p, model_cg

## The cell below is **NOT** to be removed
##### The function is to be amended so that it accepts the given input (dataframe) and returns the required output (list).
##### It is recommended to test the function out prior to submission
-------------------------------------------------------------------------------------------------------------------------------
##### The hidden_data parsed into the function below will have the same layout columns wise as the dataset *SENT* to you
##### Thus, ensure that steps taken to modify the initial dataset to fit into the model are also carried out in the function below

In [61]:
def testing_hidden_data(hidden_data: pd.DataFrame) -> list:
    '''DO NOT REMOVE THIS FUNCTION.

The function accepts a dataframe as input and return an iterable (list)
of binary classes as output.

The function should be coded to test on hidden data
and should include any preprocessing functions needed for your model to perform.

All relevant code MUST be included in this function.'''

    # apply feature engineering to data
    df_p, df_cg = feature_engineering(hidden_data)
    X_test_p = df_p
    X_test_cg = df_cg

    # load model
    model_p, model_cg = load_model()

    # make predictions
    y_pred_p = model_p.predict(X_test_p)
    y_pred_cg = model_cg.predict(X_test_cg)

    # combine the two predictions
    p_pred_ind = pd.DataFrame({'index': df_p.index, 'y_pred': y_pred_p})
    cg_pred_ind = pd.DataFrame({'index': df_cg.index, 'y_pred': y_pred_cg})
    combined_pred_ind = pd.concat([p_pred_ind, cg_pred_ind])
    orig_ind = pd.DataFrame({'index': hidden_data.index})
    orig_pred = pd.merge(orig_ind, combined_pred_ind, on='index', how='left')
    orig_pred = orig_pred.sort_values(by=['index'])
    orig_pred = orig_pred.set_index('index')
    orig_pred = orig_pred.fillna(0)

    # convert pandas series to list
    result = orig_pred['y_pred'].tolist()

    return result

##### Cell to check testing_hidden_data function

In [62]:
# This cell should output a list of predictions.
test_df = pd.read_parquet(filepath)
test_df = test_df.drop(columns=["f_purchase_lh"])
print(testing_hidden_data(test_df))

### Please have the filename renamed and ensure that it can be run with the requirements above being met. All the best!